<a href="https://colab.research.google.com/github/vuhpham94/nflx-adv-data-hw/blob/main/11122021/01-SkillDrill/Solved/ZillowHousingPrices_SparkSQL-VP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-16 13:37:19--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.15MB/s    in 0.2s    

2021-11-16 13:37:20 (5.15 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Zillow Housing Prices").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/unit-7/zillow_housing_prices.csv"


In [5]:
spark.sparkContext.addFile(url)
housing_data = spark.read.option('header', 'true').csv(SparkFiles.get("zillow_housing_prices.csv"), inferSchema=True, sep=',')

In [6]:
housing_data.show(5)

+---+--------+-----------+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+

In [7]:
#Create a temp view of your dataframe
housing_data.createOrReplaceTempView('housing')

In [8]:
# Cast 2020-01 column as float and display the average price per state using Spark SQL
spark.sql("""
select StateName, avg(cast(`2020-01` as float)) as avg_2020_01
from housing
group by StateName
""").show(10)

+------------+------------------+
|   StateName|       avg_2020_01|
+------------+------------------+
|        Utah|          345200.0|
|      Hawaii|         572868.75|
|   Minnesota|256823.59550561797|
|        Ohio| 179073.9393939394|
|      Oregon| 342466.6666666667|
|    Arkansas|162611.42857142858|
|       Texas|266331.35135135136|
|North Dakota|230916.66666666666|
|Pennsylvania|246516.66666666666|
| Connecticut|291745.28301886795|
+------------+------------------+
only showing top 10 rows



In [9]:
# Repeat the same cast and average price for the month of February or 2020-02 using SparkSQL
spark.sql("""
select StateName, avg(cast(`2020-02` as float)) as avg_2020_02
from housing
group by StateName
""").show(10)

+------------+------------------+
|   StateName|       avg_2020_02|
+------------+------------------+
|        Utah|  347562.962962963|
|      Hawaii|          592525.0|
|   Minnesota|261769.76744186046|
|        Ohio|182818.30065359478|
|      Oregon|348658.82352941175|
|    Arkansas|165128.57142857142|
|       Texas| 269829.6703296703|
|North Dakota|          232660.0|
|Pennsylvania| 250119.6261682243|
| Connecticut| 297518.8679245283|
+------------+------------------+
only showing top 10 rows



In [10]:
# Repeat the same cast and average price per state for the month of February, but filter out any states with greater than $200000 average price
out_df = spark.sql("""
select StateName, avg(cast(`2020-02` as float)) as avg_2020_02
from housing
group by StateName
having avg_2020_02 > 200000
""")
out_df.show(10)

+------------+------------------+
|   StateName|       avg_2020_02|
+------------+------------------+
|        Utah|  347562.962962963|
|      Hawaii|          592525.0|
|   Minnesota|261769.76744186046|
|      Oregon|348658.82352941175|
|       Texas| 269829.6703296703|
|North Dakota|          232660.0|
|Pennsylvania| 250119.6261682243|
| Connecticut| 297518.8679245283|
|      Nevada|          327770.0|
|  Washington| 439433.3333333333|
+------------+------------------+
only showing top 10 rows



In [11]:
# Convert your SparkSQL dataframe into a Pandas dataframe and save your Pandas dataframe into a CSV file
out_df.toPandas().to_csv('avg_housing_prize_022020_bystate_greater_200k.csv')